# Loading

In [1]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import openai
import string
import json
import copy

In [2]:
client = OpenAI(api_key="", base_url="https://api.deepseek.com")
model="deepseek-chat" # DeepSeek-V3

# Seniority

In [3]:
train_df = pd.read_csv('data/seniority_labelled_development_set_cleaned.csv')
test_df = pd.read_csv('data/seniority_labelled_test_set_cleaned.csv')

In [4]:
train_df.iloc[1131]

job_id                                                             33027740
job_title                                                   Fitter & Welder
job_summary               A food manufacturing company has an opportunit...
job_ad_details            A well known food manufacturer operating since...
classification_name                    Manufacturing, Transport & Logistics
subclassification_name                                                Other
y_true                                                          experienced
Name: 1131, dtype: object

# RBIC Functions

### RB


In [5]:
#== step 1: role explanation
def step_1():
    messages = [
        {"role": "system", "content": "You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label."},
        {"role": "user", "content": "Based on your role, can you briefly explain what seniority means, and what seniority labels look like?"}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    return response.choices[0].message.content, messages

In [6]:
#== few-shot examples
def fewshot(messages):
    # messages.append({
    #     "role": "user",
    #     "content": "I will provide you with some examples on how to accomplish your task"
    # })

    # response = client.chat.completions.create(
    #     model=model,
    #     messages=messages
    # )
    # messages.append({"role": "assistant", "content": response.choices[0].message.content})

    few_shot_indices = [2081,1,5,12,3]

    for i in few_shot_indices:
        desc = {
          "job_title": train_df.iloc[i].job_title,
          "job_summary": train_df.iloc[i].job_summary,
          "job_ad_details": train_df.iloc[i].job_ad_details,
          "classification_name": train_df.iloc[i].classification_name,
          "subclassification_name": train_df.iloc[i].subclassification_name
        }
        desc_str = str(desc)

        # add the description
        messages.append({
          "role": "user",
          "content": desc_str
        })

        label = train_df.iloc[i].y_true
        label_str = str(label)

        # add the output
        messages.append({
            "role": "assistant",
            "content": label_str
        })

    return messages

In [7]:
#== step 2: setting sub-task --> ask for seniority patterns
def step_2(messages):
    messages.append({
        "role": "user",
        "content": "As a seniority label predictor, what are some common phrases or patterns that indicate a seniority label in a job description?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    return response.choices[0].message.content, messages

### IC

In [8]:
#== step 3: presence of seniority (skipped)
def step_3(messages_static, desc_str):
    messages = copy.deepcopy(messages_static)

    messages.append({
        "role": "user",
        "content": f"{desc_str} does this job description include any seniority-related information? Just respond with 'Yes' or 'No'."
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    p3_content = response.choices[0].message.content

    # clean and check the response
    response_p3 = p3_content.translate(str.maketrans('', '', string.punctuation))
    response_p3 = response_p3.strip().lower()[:3]
    #print(f"step3 messages: {len(messages)}")
    return response_p3, messages

In [9]:
#== step 4: iterative coaching/finding clues to prevent hallucination
def step_4(response_p3, messages):
    messages.append({
            "role": "user",
            "content": "Extract the seniority-related phrases from the text verbatim. Respond in JSON: {\"Clue\": \"\"}."
        })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    #print(f"step4 messages: {len(messages)}")

    return response.choices[0].message.content, messages

In [10]:
#== step 5: use the clue to generate the final output
def step_5(messages):
    messages.append({
        "role": "user",
        "content": (
            "Based on the extracted seniority clue, return a structured seniority label in the format {\"seniority_label\": \"\"}. "
                        
        )
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    answer_str = response.choices[0].message.content
    
    # format and print the output
    try:
        answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
        answer_str_ = answer_str_.replace('“', '"')
        answer_str_ = answer_str_.replace('”', '"')
        answer = json.loads(answer_str_)
        
        label = answer['seniority_label']
    except ...:
        print(f"Failed to parse model output: {answer_str}")
        label = "ERROR " + answer_str

    return label, answer_str

In [11]:
def RBIC_static_messages(verbose=False, add_fewshot=True):
    response, messages = step_1()
    if verbose: print(f"RB step 1: {response}\n")

    response, messages = step_2(messages)
    if verbose: print(f"RB step 2: {response}\n")

    if add_fewshot:
        messages = fewshot(messages)
        if verbose: print(f"Fewshot examples added\n")
    return messages

In [12]:
def RBIC(messages, desc_str, verbose=False):
    response_p3, messages_local = step_3(messages, desc_str)
    if verbose: print(f"IC step 1: {response_p3}\n")

    response, messages_local = step_4(response_p3, messages_local)
    if verbose: print(f"IC step 2: {response}\n")

    label, answer_str = step_5(messages_local)
    if verbose: print(f"IC step 3 (Final): {label}\n")
    if verbose: print(f"IC step 3 (Final Raw): {answer_str}\n")

    return str(label)

# Testing

### Qualitative Tests

In [13]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=True)

RB step 1: Certainly!  

**Seniority** refers to the level of experience, responsibility, and authority associated with a job role. It helps categorize positions based on career progression, from entry-level roles to executive leadership.  

### **Common Seniority Labels (Hierarchy from Junior to Senior)**  
1. **Entry-Level / Junior / Trainee / Apprentice / Graduate** – Little to no prior experience required.  
2. **Intermediate / Mid-Level / Associate / Standard** – Some experience, independent work expected.  
3. **Senior / Experienced / Specialist / Supervisor** – Advanced expertise, may mentor others.  
4. **Lead / Manager / Assistant Manager / Coordinator** – Team or project leadership.  
5. **Head / Director / Chief / Principal / Executive** – Strategic decision-making, high responsibility.  
6. **Board / Owner / C-Level (CEO, CFO, etc.)** – Top leadership or ownership roles.  

### **Other Variations**  
- **Hybrid Labels:** (e.g., "Senior/Lead," "Junior-Intermediate")  
- **Ex

In [14]:
ind = 28

#[2081,1,5,12,3]

desc = {
    "job_title": train_df.iloc[ind].job_title,
    "job_summary": train_df.iloc[ind].job_summary,
    "job_ad_details": train_df.iloc[ind].job_ad_details,
    "classification_name": train_df.iloc[ind].classification_name,
    "subclassification_name": train_df.iloc[ind].subclassification_name,
  }
desc_str = str(desc)

label = train_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 15
IC step 1: yes

IC step 2: ```json
{
  "Clue": "This position leads and supports a team of staff",
  "Clue": "Leading a team to ensure development of local area plans",
  "Clue": "Creating and maintaining a culture consistent with the organisation’s values",
  "Clue": "Leading and ensuring the development of any policies, standards, procedures and strategic documents",
  "Clue": "Demonstrable experience and an excellent reputation in the leadership and management of people within a comparable organisation"
}
```

IC step 3 (Final): manager

IC step 3 (Final Raw): ```json
{
  "seniority_label": "manager"
}
```

pred = manager
truth = senior


### Quantitative Tests

In [15]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [16]:
test_pred_df

,y_pred


In [17]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_summary": test_df.iloc[i].job_summary,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "classification_name": test_df.iloc[i].classification_name,
        "subclassification_name": test_df.iloc[i].subclassification_name,
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)

    label_pred = RBIC(messages_static, desc_str)

    test_pred_df.loc[len(test_pred_df)] = label_pred

# export the dataframe to a new csv file
test_pred_df.to_csv('seniority_labelled_test_set_deepseek_rbic_fewshot_preds.csv', index=False)

100%|███████████████████████████████████████| 689/689 [2:57:37<00:00, 15.47s/it]


# No few-shot

In [18]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=False)

RB step 1: Certainly! **Seniority** in job descriptions refers to the level of experience, responsibility, and authority associated with a role. It helps candidates understand whether a position is suitable for their career stage (e.g., a recent graduate vs. a seasoned professional).  

### **Common Seniority Labels (Hierarchy Approximate):**  
1. **Entry-Level / Junior / Graduate / Trainee / Apprentice** – Little to no prior experience required; training provided.  
2. **Intermediate / Associate / Mid-Level** – Some experience (2–5 years), moderate autonomy.  
3. **Senior / Experienced / Specialist** – Significant expertise (5+ years), leadership in tasks.  
4. **Lead / Supervisor / Assistant Manager** – Oversees small teams or projects.  
5. **Manager / Head / Director / Principal** – Strategic leadership, department-level decisions.  
6. **Executive / Chief (C-level) / Board / Owner** – Top-tier leadership (CEO, CFO, etc.).  

### **Variations & Nuances:**  
- **"Senior/Lead"** – Hy

In [19]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])
test_pred_df

,y_pred


In [20]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_summary": test_df.iloc[i].job_summary,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "classification_name": test_df.iloc[i].classification_name,
        "subclassification_name": test_df.iloc[i].subclassification_name,
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)

    label_pred = RBIC(messages_static, desc_str)

    test_pred_df.loc[len(test_pred_df)] = label_pred
    
test_pred_df.to_csv('seniority_labelled_test_set_deepseek_rbic_preds.csv', index=False)

100%|███████████████████████████████████████| 689/689 [3:17:54<00:00, 17.23s/it]


# Regular prompting

In [21]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [22]:
messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label."},
]

In [23]:
for i in tqdm(range(len(test_df))):
    desc = {
      "job_title": test_df.iloc[i].job_title,
      "job_summary": test_df.iloc[i].job_summary,
      "job_ad_details": test_df.iloc[i].job_ad_details,
      "classification_name": test_df.iloc[i].classification_name,
      "subclassification_name": test_df.iloc[i].subclassification_name
    }
    desc_str = str(desc)

    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract seniority label from this job description. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label. "
            "Respond in JSON: {\"seniority_label\": \"\"}."
        )
    })
    
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    answer_str = response.choices[0].message.content

    # format and print the output
    try:
        answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
        answer_str_ = answer_str_.replace('“', '"')
        answer_str_ = answer_str_.replace('”', '"')
        answer = json.loads(answer_str_)
        
        label = answer['seniority_label']
    except ...:
        print(f"Failed to parse model output: {answer_str}")
        label = "ERROR " + answer_str

    if type(label) == list:
        label = '/'.join(label)
    test_pred_df.loc[len(test_pred_df)] = label

# export the dataframe to a new csv file
test_pred_df.to_csv('seniority_labelled_test_set_deepseek_preds.csv', index=False)

100%|█████████████████████████████████████████| 689/689 [57:45<00:00,  5.03s/it]


# Metrics

In [1]:
import json
import string
import pandas as pd
import numpy as np
import ast

In [2]:
def categories(label):
    lab = str(label).lower()
    if 'entry' in lab:
        return 'Entry'
    elif 'junior' in lab or 'assistant' in lab:
        return 'Junior'
    elif 'intermediate' in lab or 'experienced' in lab or 'mid' in lab:
        return 'Mid'
    elif 'senior' in lab or 'lead' in lab:
        return 'Senior'
    elif any(x in lab for x in ['manager','director','chief','head','executive','principal']):
        return 'Leadership'
    else:
        return 'Other'
    
def process(row):
    if "error" not in row:
        row = row.strip().lower()
        if '[' in row and ']' in row:
            row = ast.literal_eval(row[row.find('['):row.find(']')+1])
            row = '/'.join(row)
        return row
    else:
        row = row.strip().lower()
        try:
            row = row[row.find('{'):row.find('}')+1]
            row_data = json.loads(row)
        except Exception:
            row = row[len('error'):]
            row = row.translate(str.maketrans('', '', string.punctuation))
            row = row.strip()
            if len(row.split(' ')) == 1:
                return row
            return row
        row_data = row_data['clue'] #if 'clue' in row_data else 
        if row_data == 'yes':
            return ""
        return row_data

def get_accuracy(path_to_preds):
    preds = pd.read_csv(path_to_preds)
    test_df = pd.read_csv('data/seniority_labelled_test_set_cleaned.csv')
    
    test_df['y_pred'] = preds.values.reshape(-1)
    test_df = test_df.fillna('')
    
    test_df['y_pred'] = test_df['y_pred'].map(process)
    test_df['y_true'] = test_df['y_true'].map(process)
    
    test_df['y_pred_cat'] = test_df['y_pred'].map(categories)
    test_df['y_true_cat'] = test_df['y_true'].map(categories)
    
    exact = (test_df['y_pred'] == test_df['y_true']).mean() * 100
    cat = ((test_df['y_pred'] != test_df['y_true']) & (test_df['y_pred_cat'] == test_df['y_true_cat'])).mean() * 100
    overall = ((test_df['y_pred'] == test_df['y_true']) | (test_df['y_pred_cat'] == test_df['y_true_cat'])).mean() * 100
    
    exact_count = (test_df['y_pred'] == test_df['y_true']).sum()
    cat_count = ((test_df['y_pred'] != test_df['y_true']) & (test_df['y_pred_cat'] == test_df['y_true_cat'])).sum()
    overall_count = ((test_df['y_pred'] == test_df['y_true']) | (test_df['y_pred_cat'] == test_df['y_true_cat'])).sum()
    
    print(f'Exact: {exact_count}/{test_df.shape[0]}')
    print(f'Similar: {cat_count}/{test_df.shape[0]}')
    print(f'Overall: {overall_count}/{test_df.shape[0]}')
    
    res = pd.DataFrame(
        {
            'Exact': round(exact, 2),
            'Similar': round(cat, 2),
            'Overall': round(overall, 2),
        },
        index=['Accuracy (%)']
    )
    
    return res

In [3]:
path_to_preds = 'seniority_labelled_test_set_deepseek_rbic_fewshot_preds.csv'
get_accuracy(path_to_preds)

Exact: 339/689
Similar: 76/689
Overall: 415/689


,Exact,Similar,Overall
Accuracy (%),49.2,11.03,60.23


In [4]:
path_to_preds = 'seniority_labelled_test_set_deepseek_rbic_preds.csv'
get_accuracy(path_to_preds)

Exact: 360/689
Similar: 99/689
Overall: 459/689


,Exact,Similar,Overall
Accuracy (%),52.25,14.37,66.62


In [5]:
path_to_preds = 'seniority_labelled_test_set_deepseek_preds.csv'
get_accuracy(path_to_preds)

Exact: 381/689
Similar: 81/689
Overall: 462/689


,Exact,Similar,Overall
Accuracy (%),55.3,11.76,67.05
